In [16]:
from glob import glob

import pandas as pd
from tqdm import tqdm
import dill

In [17]:
with open("./read_csv.pkl", 'rb') as f:
    read_csv = dill.load(f)
    
with open("./preproc_column.pkl", 'rb') as f:
    preproc_column = dill.load(f)

In [18]:
# Params
BASE_COLS = ['사업자등록번호', '자료생성년월']

ADDR_COLS = [
    '사업장지번상세주소', 
    '사업장도로명상세주소',
    '고객법정동주소코드', 
    '고객행정동주소코드',
    '법정동주소광역시도코드',
    '법정동주소광역시시군구코드',
    '법정동주소광역시시군구읍면동코드',
    '우편번호',
]

CORP_COLS = [
    '사업장명',
    '사업장가입상태코드',
    '사업장형태구분코드',
    '사업장업종코드',
    '사업장업종코드명',
    '적용일자',
    '재등록일자',
    '탈퇴일자',
]

NPS_COLS = [
    '가입자수',
    '당월고지금액',
    '신규취득자수',
    '상실가입자수',
]

In [19]:
# Loads
file_paths = glob("./data/raw_data/*.csv")

In [20]:
# Fuctions
##
def slice_column(df, base_cols, feature_cols):
    sliced_df = df.loc[:, base_cols + feature_cols]
    return sliced_df


slice_addr = lambda df: slice_column(df, BASE_COLS, ADDR_COLS)
slice_corp = lambda df: slice_column(df, BASE_COLS, CORP_COLS)
slice_nps = lambda df: slice_column(df, BASE_COLS, NPS_COLS)


##
def get_date(df, date_col):
    date = df[date_col].mode().iloc[0]
    return date

In [21]:
for file_path in tqdm(file_paths):

    df = read_csv(file_path)
    df.columns = list(map(preproc_column, df.columns))
    date = get_date(df, BASE_COLS[-1])
    object_columns = df.convert_dtypes().select_dtypes('object').columns
    df.loc[:, object_columns] = df.loc[:, object_columns].astype(str)
    
    slice_addr(df).to_csv(f"./data/preproc_data/addr_{date}.csv")
    slice_corp(df).to_csv(f"./data/preproc_data/corp_{date}.csv")
    slice_nps(df).to_csv(f"./data/preproc_data/nps_{date}.csv")

100%|██████████| 100/100 [06:36<00:00,  3.97s/it]
